In [ ]:
import sys
sys.path.append("..")
from LowLevelModules.XPScontroller import XPSstage
from LowLevelModules.NIdaqAPD import APDCounter
from LowLevelModules.GeneralFunctions import *
import time
import ipywidgets as widgets
from IPython.display import display
from scipy.optimize import curve_fit
import numpy as np
from sklearn.metrics import r2_score

# stage stuff
ip_address = '169.254.66.147'
portnum = 5001
groupnum = 1
stage = XPSstage(ip_address, portnum, groupnum)
stage.read_position()

In [ ]:
"""#######################################   Default parameters   #########################################"""
terminal = "/Dev1/PFI1"   # APD that has the HWP and polarizer in front
scan_extent = 2    #microns
steps = 20         
collection_time = .2   #seconds
"""#########################################################################################################"""
# initialize scan range
scan_values = stage.read_position() + np.arange(-scan_extent, scan_extent, steps)/1000

lp = LivePlot(1, 9, 4, 'o', 'x (\microm)', 'APD counts (kHz)')
pos, cts = [], []
for p in scan_values:
    try:
        # move stage to new position
        stage.move_to(p)
        # collect the APD count rate
        APD1 = APDCounter(terminal, task_name='stage scan counts')
        APD1.start()
        time.sleep(collection_time)
        APD_cts = APD1.read() / collection_time / 1000
        APD1.close()
        # get the new data and update the plot
        pos.append(p)
        cts.append(APD_cts)
        lp.plot_live(pos, cts)
    except KeyboardInterrupt:
        # press the stop button to trigger this
        APD1.close()
        break

In [ ]:
# fit data and move the stage
bds = ([.1, scan_values[0], 0.0001],[np.inf, scan_values[-1], 0.1])
popt, pcov = curve_fit(gaussian_func, pos, cts, bounds=bds)
data_y_fit = gaussian_func(pos, *popt)
plt.plot(pos, data_y_fit, 'r-')
#check validity of the fit
if r2_score(cts, data_y_fit)>.9:
    best_position = popt[1]
else:
    best_position = pos(cts.argmax())
# move the stage
stage.move_to(best_position)